In [1]:
from src.core import strategy
from src.core import metrics
import yfinance as yf

prices = yf.download("XLU, XLK, XLB, XLP, XLY, XLI, XLV, XLF")["Adj Close"]


# strategy = strategy.Momentum(prices).simulate(
#     start="2015-1-1", min_volatility=0.12, max_volatility=0.15
# )

# strategy.value.plot()
# print(strategy.analytics())


[*********************100%***********************]  8 of 8 completed


In [3]:
from src.core import optimizer

optimizer.Optimizer(prices=prices).uniform_allocation()

XLB    0.125
XLF    0.125
XLI    0.125
XLK    0.125
XLP    0.125
XLU    0.125
XLV    0.125
XLY    0.125
Name: weights, dtype: float64

In [ ]:
import pandas as pd

meta = pd.read_excel("database.xlsx", sheet_name="tb_meta")

import yfinance as yf

# for id, m in meta.iterrows():
#     tic = yf.Ticker(m.ticker)
#     ff = tic.isin
#     print(ff)

for id, row in meta.iterrows():
    print(row.ticker)
    if row.__ticker.endswith(".KS"):
        continue

    t = yf.Ticker(row.ticker)

    hist = t.history(period="1y")
    try:
        splits = hist["Stock Splits"]

        test = (splits != 0).any()
        if test:
            print("Has splits")
            print(row.ticker)
    except:
        continue


In [ ]:
import yfinance as yf

t = yf.Ticker("HYMB")
hist = t.history(period="1y")
splits = hist["Stock Splits"]
splits = splits[splits != 0]
splits


In [ ]:
""" base strategy class """

import warnings
from typing import Any, List, Optional, Dict
import numpy as np
import pandas as pd


class Account:
    """strategy account"""

    def __init__(self):
        self.date: List = []
        self.value: List = []
        self.weights: List[Dict] = []
        self.reb_weights: List[Dict] = []
        self.trade_weights: List[Dict] = []


class BaseStrategy:
    """
    BaseStrategy class is an algorithmic trading strategy that sequentially allocates capital among
    group of assets based on pre-defined allocatioin method.

    BaseStrategy shall be the parent class for all investment strategies with period-wise
    rebalancing scheme.

    Using this class requires following pre-defined methods:
    1. rebalance(self, price_asset, date, **kwargs):
        the method shall be in charge of calculating new weights based on prices
        that are avaiable.
    2. monitor (self, ...):
        the method shall be in charge of monitoring the strategy status, and if
        necessary call the rebalance method to re-calculate weights. aka irregular rebalancing.
    """

    def __init__(
        self,
        price_asset: pd.DataFrame,
        frequency: str = "M",
        min_assets: int = 2,
        min_periods: int = 2,
        investment: float = 1000.0,
        commission: int = 0,
        currency: str = "KRW",
        name: str = "strategy",
        account: Optional[Account] = None,
    ) -> None:
        """Initialization"""
        self.price_asset: pd.DataFrame = self.check_price_df(price_asset)
        self.frequency: str = frequency
        self.min_assets: int = min_assets
        self.min_periods: int = min_periods
        self.name: str = name
        self.commission = commission
        self.currency = currency

        # account information
        self.idx: int = 0
        self.date: Any = None
        self.value: float = investment
        self.weights: pd.Series = pd.Series(dtype=float)
        self.reb_weights: pd.Series = pd.Series(dtype=float)
        self.trade_weights: pd.Series = pd.Series(dtype=float)
        self.account: Account = account or Account()

    ################################################################################################
    @property
    def value_df(self):
        """values dataframe"""
        return pd.DataFrame(
            data=self.account.value, index=self.account.date, columns=["value"]
        )

    @property
    def weights_df(self):
        """weights dataframe"""
        return pd.DataFrame(data=self.account.weights, index=self.account.date)

    @property
    def reb_weights_df(self):
        """weights dataframe"""
        return pd.DataFrame(data=self.account.reb_weights, index=self.account.date)

    @property
    def trade_weights_df(self):
        """weights dataframe"""
        return pd.DataFrame(data=self.account.trade_weights, index=self.account.date)

    ################################################################################################

    def update_book(self) -> None:
        """update the account value based on the current date"""
        prices = self.price_asset.loc[self.date]
        if not self.weights.empty:
            print(f"update book values")
            pre_prices = self.price_asset.iloc[
                self.price_asset.index.get_loc(self.date) - 1
            ]
            capitals = self.weights * self.value
            pri_return = prices / pre_prices
            new_capitals = capitals * pri_return.loc[capitals.index]
            profit_loss = new_capitals - capitals
            self.value += profit_loss.sum()
            self.weights = new_capitals / self.value

        if not self.reb_weights.empty:
            print(f"make re-allocation {self.reb_weights.to_dict()}")
            # reindex to contain the same asset.
            union_assets = self.reb_weights.index.union(self.weights.index)
            self.weights = self.weights.reindex(union_assets, fill_value=0)
            self.reb_weights = self.reb_weights.reindex(union_assets, fill_value=0)
            self.trade_weights = self.reb_weights.subtract(self.weights)
            trade_capitals = self.value * self.trade_weights
            trade_costs = trade_capitals.abs() * self.commission / 10_000
            trade_cost = trade_costs.sum()
            # update the account metrics.
            self.value -= trade_cost
            self.weights = self.reb_weights

        # do nothing if no account data.
        if self.weights.empty and self.reb_weights.empty:
            return
        # loop through all variables in account history
        for name in vars(self.account).keys():
            getattr(self.account, name).append(getattr(self, name))
        # clear the rebalancing weights.
        self.reb_weights = pd.Series(dtype=float)
        self.trade_weights = pd.Series(dtype=float)

    ################################################################################################

    def simulate(self, start: ... = None, end: ... = None) -> ...:
        """simulate historical strategy perfromance"""
        start = start or self.price_asset.index[0]
        end = end or self.price_asset.index[-1]
        reb_dates = pd.date_range(start=start, end=end, freq=self.frequency)
        for self.date in pd.date_range(start=start, end=end, freq="D"):
            print(self.date)
            if self.date in self.price_asset.index:
                self.update_book()
            if self.weights.empty or self.monitor() or self.date in reb_dates:
                if not self.reb_weights.empty:
                    continue
                self.reb_weights = self.allocate(self.date)
                print(f"rebalancing weights: {self.reb_weights.to_dict()}")
        return self

    def allocate(self, date: ... = None) -> pd.Series:
        """allocate weights based on date if date not provided use latest"""
        # pylint: disable=multiple-statements
        date = date or self.price_asset.index[-1]
        price_slice = self.price_asset.loc[:date].dropna(
            thresh=self.min_periods, axis=1
        )
        if price_slice.empty:
            return pd.Series(dtype=float)
        reb_weights = self.rebalance(price_asset=price_slice)
        if reb_weights is None:
            return pd.Series(dtype=float)
        return self.clean_weights(reb_weights, decimals=4)

    def rebalance(self, price_asset: pd.DataFrame) -> pd.Series:
        """Default rebalancing method"""
        asset = price_asset.columns
        uniform_weight = np.ones(len(asset))
        uniform_weight /= uniform_weight.sum()
        weight = pd.Series(index=asset, data=uniform_weight)
        return weight

    def monitor(self) -> bool:
        """Default monitoring method."""
        return False

    ################################################################################################
    @staticmethod
    def check_price_df(price_df: pd.DataFrame) -> pd.DataFrame:
        """Check the price_df.

        Args:
            price_df (pd.DataFrame): _description_

        Raises:
            TypeError: if price_df is not pd.DataFrame.

        Returns:
            pd.DataFrame: price_df
        """
        if not isinstance(price_df, pd.DataFrame):
            raise TypeError("price_df must be a pd.DataFrame.")
        if not isinstance(price_df.index, pd.DatetimeIndex):
            warnings.warn("converting price_df's index to pd.DatetimeIndex.")
            price_df.index = pd.to_datetime(price_df.index)
        return price_df

    @staticmethod
    def clean_weights(weights: pd.Series, decimals: int = 4) -> pd.Series:
        """Clean weights based on the number decimals and maintain the total of weights.

        Args:
            weights (pd.Series): asset weights.
            decimals (int, optional): number of decimals to be rounded for
                weight. Defaults to 4.

        Returns:
            pd.Series: clean asset weights.
        """
        # clip weight values by minimum and maximum.
        tot_weight = weights.sum().round(4)
        weights = weights.round(decimals=decimals)
        # repeat round and weight calculation.
        for _ in range(10):
            weights = weights / weights.sum() * tot_weight
            weights = weights.round(decimals=decimals)
            if weights.sum() == tot_weight:
                return weights
        # if residual remains after repeated rounding.
        # allocate the the residual weight on the max weight.
        residual = tot_weight - weights.sum()
        # !!! Error may occur when there are two max weights???
        weights[np.argmax(weights)] += np.round(residual, decimals=decimals)
        return weights


####################################################################################################
def backtest(allocation_df: pd.DataFrame, price_df: pd.DataFrame) -> BaseStrategy:
    """provide a backtested strategy.

    Args:
        allocation_df (pd.DataFrame): allocation dataframe.
        price_df (pd.DataFrame): asset price dataframe.

    Returns:
        BaseStrategy: backtested strategy.
    """

    class Backtest(BaseStrategy):
        """backtest class"""

        def rebalance(self, price_asset: pd.DataFrame) -> pd.Series:
            if self.date in allocation_df.index:
                weights = allocation_df.loc[self.date].dropna()
                return weights[weights != 0]
            return pd.Series(dtype=float)

    strategy = Backtest(price_asset=price_df, frequency="D").simulate(
        start=allocation_df.index[0]
    )

    return strategy


path = r"C:\Users\vip\OneDrive\DWS\ABL_RESULT\MLP_ALLOCATION.xlsx"

import pandas_datareader as pdr

allocation = pd.read_excel(path, "allocation", index_col=[1, 2, 0], parse_dates=True)
allocation = allocation.unstack().unstack()["weights"]

stra = "US_5"

allo = allocation[stra].dropna(how="all", axis=1)

tickers = ", ".join(allo.columns.tolist())
tickers = tickers.replace(".KS", "")
tickers = (
    tickers
    if isinstance(tickers, (list, set, tuple))
    else tickers.replace(",", " ").upper().split()
)

prices = []
if stra.startswith("KR"):
    for ticker in tickers:
        price = pdr.DataReader(ticker, "naver", start="1990-1-1").astype(float)["Close"]
        price.name = ticker + ".KS"
        prices.append(price)

    prices = pd.concat(prices, axis=1)

    prices.index = pd.to_datetime(prices.index)
if stra.startswith("US"):
    import yfinance as yf

    prices = yf.download(tickers)["Adj Close"]

result = backtest(allocation_df=allo, price_df=prices)

with pd.ExcelWriter(f"{stra}.xlsx", engine="openpyxl") as writer:
    # Save each dataframe to a separate sheet in the Excel file
    value = result.value_df
    value.index.name = "date"
    weights = result.weights_df
    weights.index.name = "date"
    al = result.reb_weights_df.dropna(how="all")
    al.index.name = "date"

    value.to_excel(writer, sheet_name="value")
    weights.to_excel(writer, sheet_name="weights")
    al.to_excel(writer, sheet_name="allocations")

value.plot()
